In [1]:
from selenium import webdriver 
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException,TimeoutException,StaleElementReferenceException
from os.path import exists
import time
import pandas as pd 
import numpy as np
import requests
from bs4 import BeautifulSoup
import re 
from datetime import date
from datetime import datetime
from datetime import timedelta
import winsound

In [3]:
def make_noise():
  duration = 2000  # milliseconds
  freq = 440  # Hz
  winsound.Beep(freq, duration)

## Helper functions

### Formatting the date:
def split_date(x):
    m = re.search(r'\d+[+]? \w+ \w+' ,str(x))
    if m:
        return m.group()
    else:
        return " "
    
def convert_date(x):
    m = re.search(r'\d+' ,str(x))
    if m:
        dt = datetime.today()-timedelta(int(m.group()))
        dt = dt.strftime('%m/%d/%Y')  #time_dict)
        return dt
    else:
        return " "
    
### Finding Zip and City
def find_zip(x):
    z = re.search(r'\d+',x )
    if z:
         return(z.group())
    else :
        return " "

## Indeed 

In [4]:
def scrape_indeed_jobs(keyword, location, n_pages):

    url = f"https://www.indeed.com/?from=gnav-homepage"
    ## Create an instance of the chrome webdriver. This is to interact with the website and test my code
    s = Service ('C:\Program Files (x86)\chromedriver_win32\chromedriver.exe')
    wd = webdriver.Chrome(service = s)
    wait = WebDriverWait(wd, 1)

    ## Open Indeed Webpage 
    wd.get(url)

    ## Enter search parameters
    what_search = wd.find_element(By.ID, 'text-input-what')
    what_search.send_keys(keyword)

    where_search = wd.find_element(By.ID, 'text-input-where')
    where_search.send_keys(Keys.CONTROL + "a")
    where_search.send_keys(Keys.DELETE)
    where_search.send_keys(location)
    where_search.send_keys(Keys.RETURN)

    base_url = wd.current_url

    ## Initialise the different lists:
    job_id =  []
    job_title = []
    seniority = []
    emp_type=[]
    job_link = []
    industries = []
    company_names = []
    company_names2 = []
    job_date =[]
    job_location = []
    job_salary = []
    job_qual = []
    job_description = []
    
    def extract(wait,job_id,job_title,seniority,emp_type,job_link,industries,company_names,company_names2,job_date,job_location,job_salary,job_qual,job_description):
        i =1
        results = wait.until(
                        EC.presence_of_element_located((By.ID, "mosaic-provider-jobcards")))
        jobs = results.find_elements(By.TAG_NAME,"a")
        
        for job in jobs:
            try:
                href = job.get_attribute("href")
                job_link.append(href)
                #print(href)
            except(NoSuchElementException, TimeoutException,StaleElementReferenceException): 
                job_link.append('')
                #print('fail')

            try :
                ids = job.get_attribute("id")
                #print(ids)
                job_id.append(ids)
            except (NoSuchElementException, TimeoutException,StaleElementReferenceException): 
                job_id.append('')

            try:
                date = job.find_element(By.XPATH, '//span[@class = "date"]').text
               # print('date', date)
                job_date.append(date)
            except (NoSuchElementException, TimeoutException,StaleElementReferenceException): 
                job_date.append("")
            
            try:
                company = job.find_element(By.XPATH,'//a[@data-tn-element = "companyName"]').get_attribute('text')
                company_names.append(company) 
                #print(company)
            except (NoSuchElementException, TimeoutException,StaleElementReferenceException): 
                company_names.append('')
            
            try:
                loc = job.find_element(By.XPATH, '//div[@class = "companyLocation"]').text
                #print(f'location, {loc}')                               
                job_location.append(loc)
            except (NoSuchElementException, TimeoutException,StaleElementReferenceException): 
                job_location.append('')
                print('location fail')

            try:
                results.find_element(By.XPATH,f'//*[@id="mosaic-provider-jobcards"]/ul/li[{i}]/div').click()
                time.sleep(.5)
                iframe =results.find_element(By.XPATH,'//*[@id="vjs-container"]').find_element(By.CSS_SELECTOR,'iframe')
                wd.switch_to.frame(iframe)
                ## Find the job's title 
                try:
                    title = wait.until(EC.presence_of_element_located((By.TAG_NAME,"h1"))).get_attribute("textContent")
                    title = title.replace(" - job post", "")
                    job_title.append(title)
                    #print(title)   
                except (NoSuchElementException, TimeoutException,StaleElementReferenceException): 
                    job_title.append('')

                try:
                    salary  = wait.until(
                                EC.presence_of_element_located((By.XPATH,'//*[@id="jobDetailsSection"]/div[2]/span'))).get_attribute('innerText')
                    job_salary.append(salary)
                    #print(f'salary,{salary}')
                except (NoSuchElementException, TimeoutException,StaleElementReferenceException): 
                    job_salary.append('')

                try:
                    emp = wait.until(
                                EC.presence_of_element_located((By.XPATH, '//*[@id="jobDetailsSection"]/div[3]/div[2]'))).get_attribute('innerText')             
                    emp_type.append(emp)
                    #print("employment type  , ", emp)
                except (NoSuchElementException, TimeoutException,StaleElementReferenceException): 
                    emp_type.append("")

                try:
                    qual = wait.until(
                                EC.presence_of_element_located((By.CLASS_NAME, "jobsearch-ReqAndQualSection-item--wrapper"))).get_attribute('innerText')
                    #print('qual',qual)
                    job_qual.append(qual)
                except (NoSuchElementException, TimeoutException,StaleElementReferenceException):
                    job_qual.append('')

                try:
                    summary = wait.until(
                                EC.presence_of_element_located((By.XPATH,'//*[@id="jobDescriptionText"]'))).get_attribute('innerText')
                    #print(f'summary,{summary}')
                    job_description.append(summary)
                except (NoSuchElementException, TimeoutException,StaleElementReferenceException):
                    job_description.append('')

                wd.switch_to.default_content()      
            except:
                job_title.append('')
                emp_type.append('')
                job_salary.append('')
                job_qual.append('')
                job_description.append('') 

            i+=1
        return job_id,job_title,seniority,emp_type,job_link,industries,company_names,company_names2,job_date,job_location,job_salary,job_qual,job_description
    
    ## Iterate over the different pages 
    i=1
    count = 1

    try:
        wait.until(EC.element_to_be_clickable((By.ID,'popover-x'))).click()       
    except (NoSuchElementException, TimeoutException,StaleElementReferenceException): 
        print("no pop up")

    job_id,job_title,seniority,emp_type,job_link,industries,company_names,company_names2,job_date,job_location,job_salary,job_qual,job_description = extract(wait,job_id,job_title,seniority,emp_type,job_link,industries,company_names,company_names2,job_date,job_location,job_salary,job_qual,job_description)
    while i<= n_pages:
        wd.execute_script("window.scrollTo(0,document.body.scrollHeight);")
        try:
            #time.sleep(1)
            button = wd.find_element(By.XPATH,f'//*[@id="resultsCol"]/nav/div/ul/li[{i}]/a').click()
            try:
                wait.until(EC.element_to_be_clickable((By.ID,'popover-x'))).click()       
            except (NoSuchElementException, TimeoutException,StaleElementReferenceException): 
                print("no pop up")
            job_id,job_title,seniority,emp_type,job_link,industries,company_names,job_date,job_location,job_salary,job_qual,job_description = extract(wait,job_id,job_title,seniority,emp_type,job_link,industries,company_names,company_names2,job_date,job_location,job_salary,job_qual,job_description)
            time.sleep(1)
            #wd.back()
            count += 1
        except:
            print("End of pages scraped")
        i+=1

    ## Cleaning the dataframe 
    job_list = {'ID' : job_id,
                'Title': job_title,
                'Date': job_date, 
                'Company': company_names,
                #'Company2': company_names2,
                'Location': job_location,
                'Salary': job_salary,
                #'Seniority': seniority,
                'Employment Type': emp_type,
                #'Function': job_func,
                #'Industry': industries,
                'Qualification': job_qual,
                'Job description':job_description,
                'Link': job_link}

    df = pd.DataFrame(job_list)
    df = df.replace('',np.nan)
    df= df.dropna(subset=['Title'])
    df['City'] = df['Location'].apply(lambda x: str(x).split(',')[0])
    df['Date Posted'] = df['Date'].apply(lambda x: convert_date(x))    
    df['Date'] = df['Date'].apply(lambda x: split_date(x))


    ## Homogenize the data frame:
    df['Zip Code'] = np.nan
    df['Seniority'] = np.nan
    df['Function'] = np.nan
    df['Industry'] = np.nan
    df['Source'] = 'Indeed'
    

    cols = ['Title', 'Date', 'Date Posted', 'Company','Location',
                'Zip Code' ,'Salary','Employment Type', 'Qualification', 'Seniority', 'Function', 'Industry','Job description', 'Link', 'Source' ]
    df= df[cols]
    return df

## Monster

In [51]:
def scrape_monster_jobs(keyword, location, n_pages):
    url = f"https://www.monster.com/jobs"

    ## Create an instance of the webdriver 
    s = Service('C:\Program Files (x86)\chromedriver_win32\chromedriver.exe')
    chrome_options = Options()
    chrome_options.add_argument( "--window-size=1000,1100")
    wd = webdriver.Chrome(service = s, options = chrome_options)

    ## Open the Monster Webpage 
    wd.get(url)
    wait = WebDriverWait(wd,2)

    ## Input the Job title and location
    what_search = wd.find_element(By.ID, "search-job" )
    what_search.send_keys(keyword)
    where_search = wd.find_element(By.ID, "search-location")
    where_search.send_keys(location)
    where_search.send_keys(Keys.RETURN)
    time.sleep(1)

    def infinite_scroll():  
        SCROLL_PAUSE_TIME = 1
        # Get scroll height
        last_height = wd.execute_script("return document.body.scrollHeight")

        while True:
            # Scroll down to bottom
            wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait to load page
            time.sleep(SCROLL_PAUSE_TIME)

            # Calculate new scroll height and compare with last scroll height
            new_height = wd.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

    i=1
    infinite_scroll()
    wd.execute_script("window.scrollTo(0,900);")
    while (i <= n_pages):
        infinite_scroll()
        wait.until(EC.presence_of_element_located((By.XPATH,'//button[@class ="sc-eCImPb dVaZLw  ds-button"]'))).click()  
        i+=1

    wd.switch_to.window(wd.window_handles[0])
    jobs = wd.find_element(By.XPATH,'//*[@id="__next"]/div[3]/main/div[2]/nav/section[1]/div[2]/div/div/div').find_elements(By.CSS_SELECTOR,'div')
    n_jobs = len(jobs)

    time.sleep(1)
    job_id =  []
    job_title = []
    seniority = []
    emp_type=[]
    job_link = []
    industries = []
    company_names = []
    job_date =[]
    job_location = []
    job_salary = []
    job_qual = []
    job_description = []
    act_url = wd.current_url
    jobs = wd.find_element(By.XPATH,'//*[@id="__next"]/div[3]/main/div[2]/nav/section[1]/div[2]/div/div/div').find_elements(By.CSS_SELECTOR,'div')
    n_jobs = len(jobs)
    wd.execute_script("window.scrollTo(0,document.body.scrollHeight);")
    print("len_jobs", n_jobs)

    for i in range(1,n_jobs):         
        try:
            job = wd.find_element(By.XPATH, f'//*[@id="__next"]/div[3]/main/div[2]/nav/section[1]/div[2]/div/div/div/div[{i}]').click()
            wd.switch_to.window(wd.window_handles[1])
            try:
                title = wd.find_element(By.TAG_NAME, "h1").text
                job_title.append(title)
                #print(title)
            except:
                job_title.append('')
            try:
                date = wd.find_element(By.XPATH, "//div[@data-test-id='svx-jobview-posted']").text
                #print(date)
                job_date.append(date)
            except:
                job_date.append('')
            try:
                company = wd.find_element(By.TAG_NAME, "h2").text
                company_names.append(company)
            except:
                company_names.append('')
                #print(company)
            try:
                location =  wd.find_element(By.XPATH, "//div[@data-test-id = 'svx-jobview-location']").text
                job_location.append(location)
            except:
                job_location.append('')
                    #print(location)
            try:
                link = wd.find_element(By.CSS_SELECTOR, 'a').get_attribute('href')
                job_link.append(link)
            except:
                    job_link.append('')
                    print(link)
            try:
                salary= wd.find_element(By.XPATH, "//span[@data-testid ='svx_salaryComponent_body']").text
                #salary = wd.find_element(By.CLASS_NAME, "salarystyle__SalaryBody-sc-1kub5et-8 jeLUTC").text
                job_salary.append(salary)
            except:
                job_salary.append('')

            try:
                emp =wd.find_element(By.XPATH, "//div[@data-test-id='svx-jobview-employmenttype']").text
                emp_type.append(emp)
            except:
                emp_type.append('')

            try:
                description = wd.find_element(By.XPATH,'//*[@id="jobview-container"]/div[1]/div/div[2]/div').text
                job_description.append(description)
                #print(description)
            except:
                job_description.append('')


            wd.close() 
            wd.switch_to.window(wd.window_handles[0]) 
            time.sleep(1)

        except:
            print('fail')
            pass
        
     ## Clean out the dataframe 
    job_list = {#'ID' : job_id,
                'Title': job_title,
                'Date': job_date, 
                'Company': company_names,
                'Location': job_location,
                'Salary': job_salary,
                #'Seniority': seniority,
                'Employment Type': emp_type,
                #'Function': job_func,
                #'Industry': industries,
                #'Qualification': job_qual,
                'Job description':job_description,
                'Link': job_link}
    df = pd.DataFrame(job_list)

     ## Remove empty lines 
    df = df.replace('',np.nan)
    df= df.dropna(subset=['Title'])

    # Convert the date
    df['Date Posted'] = df['Date'].apply(lambda x: convert_date(x))
    df['Date'] = df['Date'].apply(lambda x: split_date(x))

    ## Add variables that don't exsist in the website 
    df['ID'] = np.nan
    df['Function'] = np.nan
    df['Industry']= np.nan
    df['Qualification'] = np.nan
    df['Seniority'] = np.nan
    df['Industry'] = np.nan
    df['Zip Code'] = np.nan
    df['Company2'] = np.nan
    df['Source'] = 'Monster'
    
    cols = [ 'Title', 'Date', 'Date Posted', 'Company', 'Company2','Location',
            'Zip Code' ,'Salary','Employment Type', 'Qualification', 'Seniority', 'Function', 'Industry','Job description', 'Link','Source' ]
    df= df[cols]
    
    return df

## Linkedin

In [64]:
def scrape_linkedin_jobs(keyword,place,n_pages):
    s = Service ('C:\Program Files (x86)\chromedriver_win32\chromedriver.exe')
    wd = webdriver.Chrome(service = s)
    wait = WebDriverWait(wd, 1)

    ## change User Agent at each request 
    username = "agneselimbi@yahoo.fr" 
    password = "Voldemort_1961" # don't forget to remove when handing over 
    url = "https://www.linkedin.com/"
    wd.get(url)


    ## Sign_in in LinkedIn 
    signin = wd.find_element(By.XPATH, '/html/body/nav/div/a[2]').click()
    ids = wd.find_element(By.ID, "username")
    ids.send_keys (username)
    code =  wd.find_element(By.ID, "password") 
    code.send_keys(password)
    code.send_keys(Keys.RETURN)
    wd.find_element(By.XPATH,'//*[@id="ember19"]').click()
    time.sleep(1)
    what_search = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "input[id^='jobs-search-box-keyword']")))
    what_search.send_keys(keyword)
    time.sleep(1)
    where_search =  wd.find_element(By.CSS_SELECTOR, "input[id^= 'jobs-search-box-location-id']")
    time.sleep(1)
    where_search.send_keys(place)
    what_search.send_keys(Keys.RETURN)
    time.sleep(1)

    ## Initialize the different lists
    job_id = []
    job_title = []
    company = []
    job_location = []
    job_salary = []
    job_date = []
    job_link = []
    job_description = []
    seniority = []
    emp_type=[]
    job_func = []
    industries = []
    company_names = []

    ## Iterate over the number of pages:
    page = 1   
    url = wd.current_url
    while page <= n_pages:       
        try: 
            wd.maximize_window()
            time.sleep(1)
            wd.execute_script("window.scrollTo(0,document.body.scrollHeight);")
            time.sleep(1)
            wait.until(EC.presence_of_element_located((By.XPATH,f'/html/body/div[6]/div[3]/div[3]/div[2]/div/section[1]/div/div/section/div/ul/li[{page}]/button'))).click()  
            print(f"page {page}")

            ## Find the number of jobs on one page:
            job_list = wd.find_element(By.CLASS_NAME,'jobs-search-results__list')
            jobs = job_list.find_elements(By.CLASS_NAME,'jobs-search-results__list-item')
            n_jobs = len(jobs)

            i= 1
            for job in jobs:
                print(i)
                url = wd.current_url
                try:
                    ids = job.get_attribute("data-job-id")
                    job_id.append(ids)
                except:
                    job_id.append('')
                    print('fail')

                try:
                    title = wait.until(EC.presence_of_element_located((By.XPATH, f'/html/body/div[6]/div[3]/div[3]/div[2]/div/section[1]/div/div/ul/li[{i}]/div/div[1]/div[1]/div[2]/div[1]/a'))).text
                    job_title.append(title)
                except:
                    job_title.append('')
                    print('title fail')

                try:
                    comp= wait.until(EC.presence_of_element_located((By.XPATH, f'/html/body/div[6]/div[3]/div[3]/div[2]/div/section[1]/div/div/ul/li[{i}]/div/div[1]/div[1]/div[2]/div[2]/a'))).text
                    company_names.append(comp)
                except:
                    company_names.append('')
                    print('company fail')

                try:
                    location = wait.until(EC.presence_of_element_located((By.XPATH, f'/html/body/div[6]/div[3]/div[3]/div[2]/div/section[1]/div/div/ul/li[{i}]/div/div[1]/div[1]/div[2]/div[3]/ul/li[1]'))).text  
                    job_location.append(location)
                except:
                    job_location.append('')
                    print('location fail')

                try:
                    uid = job.get_attribute("id")
                    path = r'//*[@id = "{}"]/div/div'.format(uid)
                    wd.find_element(By.XPATH,path).click()
                    cur_url = wd.current_url
                    time.sleep(3)
                    try:
                        emp = wait.until(EC.presence_of_element_located((By.XPATH, f'/html/body/div[6]/div[3]/div[3]/div[2]/div/section[2]/div/div/div[1]/div/div[1]/div/div[2]/div[2]/ul/li[1]/span'))).text
                        emp_type.append(emp)
                    except:
                        emp_type.apppend('')    
                        print('employment fail')
                    try:                         
                        link = wait.until(EC.presence_of_element_located((By.XPATH, f'/html/body/div[6]/div[3]/div[3]/div[2]/div/section[2]/div/div/div[1]/div/div[1]/div/div[2]/a'))).get_attribute('href')
                        job_link.append(link)
                    except:
                        job_link.append('')
                        print('link fail')

                    try:
                        industry = wait.until(EC.presence_of_element_located((By.XPATH,'/html/body/div[6]/div[3]/div[3]/div[2]/div/section[2]/div/div/div[1]/div/div[1]/div/div[2]/div[2]/ul/li[2]/span'))).text
                        industries.append(industry)     
                    except:
                        industries.append('')
                        print('industry fail')

                    try:
                        age = wait.until(EC.presence_of_element_located((By.XPATH, '/html/body/div[6]/div[3]/div[3]/div[2]/div/section[2]/div/div/div[1]/div/div[1]/div/div[2]/div[1]/span[2]/span[1]'))).text
                        job_date.append(age)
                    except:
                        job_date.append('')
                        print('date fail')

                    try:
                        description = wait.until(EC.presence_of_element_located((By.XPATH,'/html/body/div[6]/div[3]/div[3]/div[2]/div/section[2]/div/div/div[1]/div/div[2]/article/div/div[1]/span'))).text
                        job_description.append(description)
                    except:
                        job_description.append('')
                        print('description fail')

                except:
                    print('fail')
                    job_link.append('')
                    job_date.append('')
                    job_description.append('')
                    emp_type.append('')
                    industries.append('')

                if cur_url != url:
                    wd.back()
                i= i+1    

        except:
            pass
        page = page+1
        #print(f'page{page}')


        ## Create a dataframe
    print(len(job_id),len(job_title),len(company_names),len(job_location),len(emp_type),len(industries),len(job_link),len(job_description))
    df = pd.DataFrame({ 'ID' : job_id,
                                 'Title': job_title,
                                 'Date Posted': job_date, 
                                 'Company': company_names,
                                 'Location': job_location,
                                 'Employment Type': emp_type,
                                 'Industry': industries,
                                 'Link': job_link,
                                 'Job description': job_description
                            })

    df['Zip Code'] = np.nan
    df['Date'] = np.nan
    df['Qualification'] = np.nan
    df['Salary'] = np.nan
    df['Source'] = 'LinkedIn'

    cols = [ 'Title', 'Date', 'Date Posted', 'Company','Location',
            'Zip Code' ,'Salary','Employment Type', 'Qualification', 'Industry','Job description', 'Link','Source']

    df= df[cols]

    return df

# Scraping files 

In [73]:
def scrape_jobs():
    ## Merge scrapped data in one file
    linkedin_df = scrape_linkedin_jobs(keyword,place,n_pages)
    indeed_df= scrape_indeed_jobs(keyword, place , n_pages)
    monster_df= scrape_monster_jobs(keyword, place , n_pages)
    make_noise()

    df = pd.concat([linkedin_df,monster_df,indeed_df])

    ## Remove the duplicates from the total dataframe 
    df = df.drop_duplicates()

    ## Export cleanead data frame to our master list 
    if exists("Master_job_list.csv"):
        df.to_csv("Master_job_list.csv", mode = 'a', index = False, header = False) 
    else:
        df.to_csv("Master_job_list.csv", index = False, header = True)   ## first run of code
    
    df

In [72]:
def update_master_df():
    master_df = pd.read_csv("Master_job_list.csv")
        
    # Remove jobs that are too old (posted more than 2 weeks ago)
    master_df = master_df[(master_df['Date Posted'] > (date.today() -timedelta(14)).strftime('%m/%d/%Y')) & (master_df['Date Posted'] <=  (date.today()+timedelta(14)).strftime('%m/%d/%Y'))]
    master_df.reset_index()    
    master_df.fillna('')

    # Discretize the job description
    def clean(text):
        # removing new line characters
        text = re.sub('\n ','',str(text))
        text = re.sub('\n',' ',str(text))
        # removing apostrophes
        text = re.sub("'s",'',str(text))
        # removing hyphensM
        text = re.sub("-",' ',str(text))
        text = re.sub("— ",'',str(text))
        # removing quotation marks
        text = re.sub('\"','',str(text))
        # removing any reference to outside text
        text = re.sub("[\(\[].*?[\)\]]", "", str(text))   
        return text.lower()

    def find_salary(txt):
        salary_pattern = re.compile(r'(\$\d+,?\d+)')
        return salary_pattern.findall(txt)

    def find_qualification(txt):
        return ([sentence + '.' for sentence in txt.split('.') if ("qualification") in sentence])

    def find_education(txt):
        return ([sentence + '.' for sentence in txt.split('.') if ("bachelor"or"master") in sentence])

    master_df['cleaned job'] = master_df["Job description"].apply(lambda x: clean(x))
    master_df['extracted salary'] = master_df['cleaned job'].apply(lambda x: find_salary(x) )
    master_df['extracted education'] = master_df['cleaned job'].apply(lambda x: find_education(x) )
    master_df['extracted qualification'] = master_df['cleaned job'].apply(lambda x: find_qualification(x) )
    
     # Save master_df
    master_df.to_csv("Cleaned_Master_job_list.csv",index = False,header =True)
    return master_df


In [76]:
## Input the keywords 
keyword = input("Enter the postions you are scraping : ")
place = input ("Enter the job's location:  ")
n_pages = int(input("Enter the number of pages you want to scrape: "))
scrape_jobs()
update_master_df()



Enter the postions you are scraping : Software Engineer
Enter the job's location:  New York, New York
Enter the number of pages you want to scrape: 1
page 1
1
2
3
description fail
4
5
6
7
8
9
10
11
12
13
14
description fail
15
16
17
18
19
20
21
22
23
24
25
25 25 25 25 25 25 25 25
no pop up
End of pages scraped
len_jobs 414
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail


NameError: name 'exists' is not defined